In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
import pandas as pd
import numpy as np
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [6]:
import nltk
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [7]:
#Printing the stopwords in english
print(stopwords.words('english'))


['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

**DATA PREPROCESSING**

In [8]:
# prompt: unzipe a file

import zipfile

# Specify the path to your zip file
zip_file_path = '/content/drive/MyDrive/Sentiment_Analyse/archive (1).zip'  # Replace with the actual path

# Specify the directory where you want to extract the files
extract_path = '/content/drive/MyDrive/Sentiment_Analyse'  # Replace with the desired extraction path

try:
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(extract_path)
    print(f"Successfully extracted files to {extract_path}")
except FileNotFoundError:
    print(f"Error: Zip file not found at {zip_file_path}")
except zipfile.BadZipFile:
    print(f"Error: Invalid zip file at {zip_file_path}")
except Exception as e:
    print(f"An unexpected error occurred: {e}")


Successfully extracted files to /content/drive/MyDrive/Sentiment_Analyse


In [9]:
#Loading the data from csv file to pandas dataframe
twitter_data = pd.read_csv('/content/drive/MyDrive/Sentiment_Analyse/training.1600000.processed.noemoticon.csv', encoding = 'ISO-8859-1')

In [10]:
#Checking the numbers of rows and columns
twitter_data.shape

(1599999, 6)

In [11]:
#Printing the first 5 rows of the coloumns
twitter_data.head()

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [12]:
#Naming the columns and reading the dataset again

coloum_names = ['target', 'ids', 'date', 'flag', 'user', 'text']
twitter_data = pd.read_csv('/content/drive/MyDrive/Sentiment_Analyse/training.1600000.processed.noemoticon.csv', encoding = 'ISO-8859-1', names = coloum_names)

In [13]:
#Checking the number of rows and coloums again
twitter_data.shape

(1600000, 6)

In [14]:
twitter_data.head()

,target,ids,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [15]:
#Counting the number of the missing values in the dataset
twitter_data.isnull().sum()

,0
target,0
ids,0
date,0
flag,0
user,0
text,0


In [16]:
#Checking the distribution of teh target coloums
twitter_data['target'].value_counts()

,count
target,
0,800000
4,800000


Convert the target "4" to "1"

In [17]:
twitter_data.replace({'target':{4:1}}, inplace=True)

In [18]:
#Checking the distribution of teh target coloums
twitter_data['target'].value_counts()

,count
target,
0,800000
1,800000


0---> Negative Tweets

1---> Positive Tweets

***Stemming*** is a process of reducing a word to its Root word

Example- actor, actress, acting = act

In [19]:
port_stem = PorterStemmer()


In [20]:
def stemming(content):
    stemmed_content = re.sub('[^a-zA-Z]', ' ', content)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]

    return stemmed_content

In [22]:
twitter_data['stemmed_content'] = twitter_data['text'].apply(stemming)

In [23]:
twitter_data.head()

,target,ids,date,flag,user,text,stemmed_content
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...","[switchfoot, http, twitpic, com, zl, awww, bum..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,"[upset, updat, facebook, text, might, cri, res..."
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,"[kenichan, dive, mani, time, ball, manag, save..."
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,"[whole, bodi, feel, itchi, like, fire]"
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....","[nationwideclass, behav, mad, see]"


In [24]:
print(twitter_data['stemmed_content'])

0          [switchfoot, http, twitpic, com, zl, awww, bum...
1          [upset, updat, facebook, text, might, cri, res...
2          [kenichan, dive, mani, time, ball, manag, save...
3                     [whole, bodi, feel, itchi, like, fire]
4                         [nationwideclass, behav, mad, see]
                                 ...                        
1599995                     [woke, school, best, feel, ever]
1599996    [thewdb, com, cool, hear, old, walt, interview...
1599997                   [readi, mojo, makeov, ask, detail]
1599998    [happi, th, birthday, boo, alll, time, tupac, ...
1599999    [happi, charitytuesday, thenspcc, sparkschar, ...
Name: stemmed_content, Length: 1600000, dtype: object


In [25]:
#Separating the data and label
X = twitter_data['stemmed_content'].values
Y = twitter_data['target'].values

In [26]:
print(X)

[list(['switchfoot', 'http', 'twitpic', 'com', 'zl', 'awww', 'bummer', 'shoulda', 'got', 'david', 'carr', 'third', 'day'])
 list(['upset', 'updat', 'facebook', 'text', 'might', 'cri', 'result', 'school', 'today', 'also', 'blah'])
 list(['kenichan', 'dive', 'mani', 'time', 'ball', 'manag', 'save', 'rest', 'go', 'bound'])
 ... list(['readi', 'mojo', 'makeov', 'ask', 'detail'])
 list(['happi', 'th', 'birthday', 'boo', 'alll', 'time', 'tupac', 'amaru', 'shakur'])
 list(['happi', 'charitytuesday', 'thenspcc', 'sparkschar', 'speakinguph', 'h'])]


In [27]:
print(Y)

[0 0 0 ... 1 1 1]


Splitting the data into train and test data

In [28]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, stratify=Y, random_state=2)

In [29]:
print(X.shape, X_train.shape, X_test.shape)

(1600000,) (1280000,) (320000,)


In [30]:
print(X_train)

[list(['watch', 'saw', 'iv', 'drink', 'lil', 'wine'])
 list(['hatermagazin'])
 list(['even', 'though', 'favourit', 'drink', 'think', 'vodka', 'coke', 'wipe', 'mind', 'time', 'think', 'im', 'gonna', 'find', 'new', 'drink'])
 ... list(['eager', 'monday', 'afternoon'])
 list(['hope', 'everyon', 'mother', 'great', 'day', 'wait', 'hear', 'guy', 'store', 'tomorrow'])
 list(['love', 'wake', 'folger', 'bad', 'voic', 'deeper'])]


In [31]:
print(X_test)

[list(['mmangen', 'fine', 'much', 'time', 'chat', 'twitter', 'hubbi', 'back', 'summer', 'amp', 'tend', 'domin', 'free', 'time'])
 list(['ah', 'may', 'show', 'w', 'ruth', 'kim', 'amp', 'geoffrey', 'sanhueza'])
 list(['ishatara', 'mayb', 'bay', 'area', 'thang', 'dammit']) ...
 list(['destini', 'nevertheless', 'hooray', 'member', 'wonder', 'safe', 'trip'])
 list(['feel', 'well']) list(['supersandro', 'thank'])]


Since The model doestnt know text data so -->> coverting textual data to numerical data

**FEATURE** **EXTRACTION**

In [32]:
# %%
#Since X_train is list of list we need to change it to string list to give TfidVectorizer

#Before applying the fit_transform convert the list to string

X_train_str = [' '.join(x) for x in X_train]
X_test_str = [' '.join(x) for x in X_test]

# %%
# FEATURE EXTRACTION
# %%
vectorizer = TfidfVectorizer()

#Use the X_train_str and X_test_str with fit_transform and transform function
X_train = vectorizer.fit_transform(X_train_str)
X_test = vectorizer.transform(X_test_str)

In [33]:
print(X_train)

  (0, 436713)	0.27259876264838384
  (0, 354543)	0.3588091611460021
  (0, 185193)	0.5277679060576009
  (0, 109306)	0.3753708587402299
  (0, 235045)	0.41996827700291095
  (0, 443066)	0.4484755317023172
  (1, 160636)	1.0
  (2, 109306)	0.4591176413728317
  (2, 124484)	0.1892155960801415
  (2, 407301)	0.18709338684973031
  (2, 129411)	0.29074192727957143
  (2, 406399)	0.32105459490875526
  (2, 433560)	0.3296595898028565
  (2, 77929)	0.31284080750346344
  (2, 443430)	0.3348599670252845
  (2, 266729)	0.24123230668976975
  (2, 409143)	0.15169282335109835
  (2, 178061)	0.1619010109445149
  (2, 150715)	0.18803850583207948
  (2, 132311)	0.2028971570399794
  (2, 288470)	0.16786949597862733
  (3, 406399)	0.29029991238662284
  (3, 158711)	0.4456939372299574
  (3, 151770)	0.278559647704793
  (3, 56476)	0.5200465453608686
  :	:
  (1279996, 318303)	0.21254698865277744
  (1279996, 434014)	0.27189450523324465
  (1279996, 390130)	0.2206474219107611
  (1279996, 373144)	0.35212500999832036
  (1279996, 23807

In [34]:
print(X_test)

  (0, 15110)	0.1719352837797837
  (0, 31168)	0.1624772418052177
  (0, 67828)	0.26800375270827315
  (0, 106069)	0.36555450010904555
  (0, 132364)	0.255254889555786
  (0, 138164)	0.23688292264071406
  (0, 171378)	0.2805816206356074
  (0, 271016)	0.45356623916588285
  (0, 279082)	0.17825180109103442
  (0, 388348)	0.2198507607206174
  (0, 398906)	0.34910438732642673
  (0, 409143)	0.3143047059807971
  (0, 420984)	0.17915624523539805
  (1, 6463)	0.30733520460524466
  (1, 15110)	0.211037449588008
  (1, 145393)	0.575262969264869
  (1, 217562)	0.40288153995289894
  (1, 256777)	0.28751585696559306
  (1, 348135)	0.4739279595416274
  (1, 366203)	0.24595562404108307
  (2, 22532)	0.3532582957477176
  (2, 34401)	0.37916255084357414
  (2, 89448)	0.36340369428387626
  (2, 183312)	0.5892069252021465
  (2, 256834)	0.2564939661498776
  :	:
  (319994, 443794)	0.2782185641032538
  (319995, 107868)	0.33399349737546963
  (319995, 109379)	0.3020896484890833
  (319995, 155493)	0.2770682832971669
  (319995, 2133

Training the machine learning model

**Logistic** **Regression**

In [35]:
model = LogisticRegression(max_iter=1000)

In [36]:
model.fit(X_train, Y_train)

LogisticRegression(max_iter=1000)

*Model* *Exaluation*

Accuracy Score

In [37]:
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(Y_train, X_train_prediction)

In [38]:
print('Accuracy score of the training data : ', training_data_accuracy)

Accuracy score of the training data :  0.79871953125


In [39]:
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(Y_test, X_test_prediction)

In [40]:
print('Accuracy score of the test data : ', test_data_accuracy)

Accuracy score of the test data :  0.77668125


Model accuracy = 77.67%

**SAVING THIS MODEL FOR FUTURE USE**

In [41]:
import pickle

In [42]:
filename = 'trained_model.sav'
pickle.dump(model, open(filename, 'wb'))

Using the saved model for the future prediction

In [43]:
#Loading the saved model
loaded_model = pickle.load(open('/content/trained_model.sav', 'rb'))

In [44]:
X_new = X_test[200]
print(Y_test[200])

prediction = model.predict(X_new)
print(prediction)

if prediction[0]==0:
  print('The tweet is Negative')
else:
  print('The tweet is Positive')

1
[1]
The tweet is Positive
